In [2]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt

### Read org data

In [3]:
df_agg = pd.read_parquet("../../data/data_agg_v0.parquet")

price_cols = [x for x in df_agg.columns if "LAST" in x]
df_subset = df_agg[["Date"] + price_cols[:12]]
df_subset.index = df_subset["Date"]
df_subset = df_subset.drop(columns=["Date"])
print(f"Shape before dropping nulls, {df_subset.shape}")
df_subset = df_subset.dropna()
print(f"Shape after dropping nulls, {df_subset.shape}")

Shape before dropping nulls, (5152, 12)
Shape after dropping nulls, (5148, 12)


In [ ]:
df_subset1 = df_subset.sort_index()

In [ ]:
mask1 = df_subset1["PX_LAST_6"]>df_subset1["PX_LAST"]
mask2 = df_subset1["PX_LAST_12"]>df_subset1["PX_LAST_6"]

df_subset1["contango"] = (mask1 & mask2).astype("int")
df_subset1["contango"].mean()

In [ ]:
contango = df_subset1[["contango"]]
for i in range(1,5):
    contango[f"lag_{i}"] = contango["contango"].shift(i)
contango["contango_start"] = ((contango.sum(axis=1)==1) & (contango["contango"]==1)).astype(int)

In [ ]:
contango["contango_start"].mean()

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
df_subset1[df_subset1.contango>0].drop(columns="contango").groupby(lambda x: (x.year)).mean().T.plot(ax=ax)

In [ ]:
pca_data = pd.read_csv("../../data/log_returns_pca.csv",index_col=0)
pca_data.index = pd.to_datetime(pca_data.index)
pca_data.shape

In [ ]:
df_merged = df_subset1[["contango"]].merge(pca_data,left_index=True,right_index=True)
df_merged[[x for x in df_merged.columns if "pca" in x\
          ]] = df_merged[[x for x in df_merged.columns if "pca" in x]].shift(1)

In [ ]:
df_merged1 = df_merged.dropna()

import statsmodels.api as sm
Xtrain = df_merged1.values
y = df_merged1["contango"]

# building the model and fitting the data
log_reg = sm.Logit(y, Xtrain).fit()

In [ ]:
Xtrain.shape

In [ ]:
print(log_reg.summary())